In [ ]:
import torch 
import argparse
from utils import dotdict
from activation_dataset import setup_token_data
import wandb
import json
from datetime import datetime
from tqdm import tqdm
from einops import rearrange
import matplotlib.pyplot as plt

cfg = dotdict()
# models: "EleutherAI/pythia-70m-deduped", "usvsnsp/pythia-6.9b-ppo", "lomahony/eleuther-pythia6.9b-hh-sft", "reciprocate/dahoas-gptj-rm-static"
cfg.model_name="reciprocate/dahoas-gptj-rm-static"
cfg.target_name="usvsnsp/pythia-6.9b-ppo"
cfg.layers=[10]
cfg.setting="residual"
# cfg.tensor_name="gpt_neox.layers.{layer}"
cfg.tensor_name="transformer.h.{layer}"
cfg.target_tensor_name="gpt_neox.layers.{layer}"
original_l1_alpha = 8e-4
cfg.l1_alpha=original_l1_alpha
cfg.sparsity=None
cfg.num_epochs=10
cfg.model_batch_size=8
cfg.lr=1e-3
cfg.kl=False
cfg.reconstruction=False
# cfg.dataset_name="NeelNanda/pile-10k"
cfg.dataset_name="Elriggs/openwebtext-100k"
cfg.device="cuda:0"
cfg.ratio = 4
cfg.seed = 0
# cfg.device="cpu"

In [ ]:
tensor_names = [cfg.tensor_name.format(layer=layer) for layer in cfg.layers]
target_tensor_names = [cfg.target_tensor_name.format(layer=layer) for layer in cfg.layers]

In [ ]:
# Load base and target autoencoders
from autoencoders.learned_dict import TiedSAE, UntiedSAE, AnthropicSAE, TransferSAE
from torch import nn



save_name = f"rm_sae_gptj"  # trim year
autoencoder = torch.load(f"trained_models/{save_name}.pt")
print(f"autoencoder loaded from f{save_name}")
autoencoder.to_device(cfg.device)

save_name = f"ppo_sae_6b" 
target_autoencoder = torch.load(f"trained_models/{save_name}.pt")
print(f"target_autoencoder loaded from f{save_name}")
target_autoencoder.to_device(cfg.device)


In [ ]:
# Initialize New transfer autoencoder
from autoencoders.learned_dict import TiedSAE, UntiedSAE, AnthropicSAE, TransferSAE
from torch import nn

modes = ["scale", "rotation", "bias", "free"]
transfer_autoencoders = []
for mode in modes:
    # mode_tsae = TransferSAE(
    #     # n_feats = n_dict_components, 
    #     # activation_size=activation_size,
    #     autoencoder,
    #     decoder=autoencoder.get_learned_dict().detach().clone(),
    #     decoder_bias=autoencoder.shift_bias.detach().clone(),
    #     mode=mode,
    # )
    mode_tsae = torch.load(f"trained_models/transfer_rm_ppo_6b_{mode}.pt")
    mode_tsae.to_device(cfg.device)
    transfer_autoencoders.append(mode_tsae)



In [ ]:
transfer_free = transfer_autoencoders[-1]